In [1]:
import numpy as np
import torch
import pickle
from mnist import load_mnist

<span style = 'font-size:1.2em;line-height:1.5em'><b>1. </b>이제까지 코드는 numpy 형식으로 되어있었습니다. 이를 Pytorch기반의 코드로 바꿔보세요. torch.nn.Linear()나 nn.ReLU()등의 High-level API를 사용하지 마시고, tensor 연산 기반의 코드로 low-level단에서 작성해보세요.</span>

- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 1. Activation function들을 Pytorch코드로 변환하셔야 됩니다. (1번문제의 결과물을 활용하셔도 되고, nn.ReLU()와 같은 pytorch에서 제공하는 함수를 사용하셔도 됩니다.)</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 2. network에 있는 numpy array를 Pytorch tensor 형태로 변환하세요.</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 3. MNIST mini-batch data는 현재 numpy array인데 Pytorch tensor 형태로 변환하세요.</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 4. Pytorch에서 텐서곱은 torch.matmul()입니다.</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 5. Pytorch에서 Tensor를 numpy array형태로 변경하는 방법은 .numpy()이다.</span>
```python
a = torch.Tensor([1,2,3])
a.numpy() # numpy array형태로 변경됨
```

- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 6. np.argmax()와 torch.argmax()는 같은 역할을 한다.</span>

```python
a = torch.rand(size=(5,3))
print(a)
# 결과값
#tensor([[0.6298, 0.9776, 0.4705],
#        [0.4715, 0.6208, 0.1938],
#        [0.5101, 0.3516, 0.7683],
#        [0.5044, 0.5985, 0.1055],
#        [0.9975, 0.6862, 0.2044]])

torch.argmax(a, dim=1)
# 결과값
# tensor([1, 1, 2, 1, 0])
```

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def sigmoid_torch(x):
    """
    return sigmoid output
    """
    result = 1/(1+torch.exp(-x))
    return result
    
def relu_torch(x):
    """
    return ReLU output
    """
    result = torch.maximum(x, torch.Tensor([0]))
    return result

def softmax1_torch(x):
    exp_sum = torch.sum(torch.exp(x))
    result = torch.exp(x)/exp_sum
    return result

def softmax2_torch(x):
    c=torch.max(x)
    exp_x = torch.exp(x-c) # overflow 대책
    exp_sum = torch.sum(exp_x)
    result = exp_x/exp_sum
    return result
    

In [4]:
def init_network(device):
    fpath = 'sample_weights/sample_weight.pkl'
    with open(fpath, 'rb') as f:
        network = pickle.load(f)
    # network변수안의 모든 key에 대해서 torch.Tensor로 형태 변경
    # GPU사용할때를 위해서 해당 tensor를 device로 옮겨놓기
    for key in network.keys():
        network[key] = torch.Tensor(network[key]).to(device)
    return network

In [5]:
def get_batch_data(X, y, start_idx, end_idx):
    x_batch = X[start_idx:end_idx]
    y_batch = y[start_idx:end_idx]
    # x_batch, y_batch를 torch.Tensor로 형태 변경
    x_batch = torch.Tensor(x_batch)
    y_batch = torch.Tensor(y_batch)
    return x_batch, y_batch

In [6]:
def forward_propagation(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    # torch의 행렬곱과 activation function을 활용하여
    # forward propagation 나타내기
    a1 = torch.matmul(x, w1) + b1
    z1 = sigmoid_torch(a1)
    a2 = torch.matmul(z1, w2) + b2
    z2 = sigmoid_torch(a2)
    a3 = torch.matmul(z2, w3) + b3
    output = softmax2_torch(a3)
    
    return output

In [7]:
(x_trn, y_trn), (x_tst, y_tst) = load_mnist(flatten=True, normalize=False)
network = init_network(device)
batch_size = 100

results = []
for i in range(0,x_trn.shape[0],batch_size):
    x_batch, _ = get_batch_data(x_trn, y_trn, i, i+batch_size)
    # x_batch를 device로 올리기 (GPU? CPU?)
    x_batch = x_batch.to(device)
    pred_score_batch = forward_propagation(network, x_batch)
    # torch의 argmax()를 활용해서 pred_score_batch
    pred_label_batch = torch.argmax(pred_score_batch, dim=1)
    # pred_label_batch를 numpy array형태로 변경
    pred_label_batch = pred_label_batch.cpu().numpy()
    results.extend(pred_label_batch)
    
print(f'미리 학습한 모델의 정확도는 {100*np.sum(results==y_trn) / len(y_trn):.2f}%입니다.')

C:\Users\user\AppData\Local\Temp\ipykernel_15572\936989516.py:5: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:205.)
  x_batch = torch.Tensor(x_batch)


미리 학습한 모델의 정확도는 92.52%입니다.


<span style = 'font-size:1.2em;line-height:1.5em'><b>2. </b>실습 파일 "3. FFNN Training (with high-level API)"의 마지막 셀에서 1-(4), 2-(3)을 보면 (line 21, 58) loss를 구할 때 F.nll_loss()를 사용한 것을 볼 수 있다. 또한, 모델을 정의한 셀에서 마지막이 log_softmax()함수를 사용한 것을 알 수 있다. 그러나, lecture note에는 최종 output layer에서 softmax를 취한 뒤, cross entropy loss를 사용하여 loss를 계산한다. 왜 이론과 실제 코드를 다르게 작성한 것일까? 이는 잘못된 코드가 아니라 pytorch에서 cross entropy loss를 계산하는 방식때문에 이렇게 코드를 작성한 것이다. 다음 사이트를 참고하여 F.nll_loss를 사용할 때와 F.cross_entropy를 사용할 때, output layer에 어떤 activation function을 취해야 하는지 설명하시오. 또한, 실습 파일 "3. FFNN Training (with high-level API)"의 내용에서 loss를 F.cross_entropy()로 구하도록 코드를 변경하시오.</span>

https://velog.io/@och9854/06-1.-Softmax-Classification

https://junstar92.tistory.com/118

In [34]:
import os, time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1. Model Class
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 1.</b> (데이터 수 * 784)크기의 행렬(X)을 네트워크에 입력</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 2.</b> 1st hidden layer를 통과.  h1 = Matmul(X, W1) + b1</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>W1: (784*50) 크기의 행렬, b1: (50, )크기의 행렬, h1: (데이터 수 * 50) 크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 3.</b> Activation함수 통과 (sigmoid)</span>
    
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 4.</b> 2nd hidden layer를 통과. h2 = Matmul(h1, W2) + b2</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>W2: (50*100) 크기의 행렬, b2 = (100, )크기의 행렬, h2: (데이터 수 * 100) 크기의 행렬</span>
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 5.</b> Activation함수 통과 (sigmoid)</span>
    
- <span style = 'font-size:1.2em;line-height:1.5em'><b>Step 6.</b> output layer를 통과. h3 = Matmul(h2, W3) + b3</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>W3: (100*10) 크기의 행렬, b3 = (10, )크기의 행렬</span>


In [35]:
class MyNet(nn.Module):
    def __init__(self, dim_in=784, dim_h1=50, dim_h2=100, dim_out=10):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(dim_in, dim_h1, bias=True)
        self.fc2 = nn.Linear(dim_h1, dim_h2, bias=True)
        self.fc3 = nn.Linear(dim_h2, dim_out, bias=True)
        
    def forward(self, x):
        h1 = self.fc1(x) # Step 1, 2
        h1 = torch.sigmoid(h1) # Step 3
        h2 = self.fc2(h1) # Step 4
        h2 = torch.sigmoid(h2) # Step 5
        out = self.fc3(h2) # Step 6
#         out = F.log_softmax(out) # Step 7
        return out

## 2. train() 함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`train()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 모델에 입력하기 적합하도록 x의 형태를 변경하고 원하는 device에 위치시키기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 지난 batch로부터 계산했던 gradient를 초기화(`zero_grad()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산. <b>여기서 loss를 `F.nll_loss()`가 아닌 `F.cross_entropy()`를 사용합니다.</b></span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Backpropagation으로 각 parameter의 gradient를 계산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> Gradient Descent로 parameter값 update</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> `trn_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 8.</b> 데이터 한 개당 평균 train loss 산출</span>

In [36]:
def train(model, data_loader, optimizer, device):
    model.train() # 모델을 학습모드로!
    trn_loss = 0
    for i, (x, y) in enumerate(data_loader):
        # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
        x = x.view(-1, 784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
        y = y.to(device)
        
        # Step 2. gradient 초기화
        optimizer.zero_grad()
        
        # Step 3. Forward Propagation
        y_pred_prob = model(x)
        
        # Step 4. Loss Calculation
        loss = F.cross_entropy(y_pred_prob, y, reduction='sum')
        
        # Step 5. Gradient Calculation (Backpropagation)
        loss.backward()
        
        # Step 6. Update Parameter (by Gradient Descent)
        optimizer.step()
        
        # Step 7. trn_loss 변수에 mini-batch loss를 누적해서 합산
        trn_loss += loss.item()
        
    # Step 8. 데이터 한 개당 평균 train loss
    avg_trn_loss = trn_loss / len(data_loader.dataset)
    return avg_trn_loss

## 3. evaluate()함수

- <span style = 'font-size:1.2em;line-height:1.5em'>`evaluate()`함수는 각 iteration마다 다음과 같이 진행됩니다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 1.</b> batch_loader로부터 mini-batch x, y 데이터를 획득하고 모델에 입력하기 적합하도록 x의 형태를 변경하고 원하는 device에 위치시키기</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 2.</b> 모델에 batch x를 입력하여 forward propagation</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 3.</b> loss function에 모델이 예측한 각 클래스에 속할 확률(`y_pred_prob`)과 실제 레이블 (`y`)을 넣어서 loss 계산. <b>여기서 loss를 `F.nll_loss()`가 아닌 `F.cross_entropy()`를 사용합니다.</b></span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 4.</b> 모델이 예측하는 레이블을 산출 (with `torch.argmax()`)</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 5.</b> Minibatch의 실제 레이블(`y`)과 예측 레이블(`y_pred_label`)을 누적하여 저장</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 6.</b> `eval_loss` 변수에 mini-batch loss를 누적해서 합산</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'><b>Step 7.</b> 데이터 한 개당 평균 evaluation loss와 accuracy 산출</span>

In [37]:
def evaluate(model, data_loader, optimizer, device):
    model.eval() # 모델을 평가모드로!
    eval_loss = 0
    
    results_pred = []
    results_real = []
    with torch.no_grad(): # evaluate()함수에는 단순 forward propagation만 할 뿐, gradient 계산 필요 X.
        for i, (x, y) in enumerate(data_loader):
            # Step 1. mini-batch에서 x,y 데이터를 얻고, 원하는 device에 위치시키기
            x = x.view(-1,784).to(device) # x.shape: [batch_size,28,28] -> [batch_size, 784]
            y = y.to(device)

            # Step 2. Forward Propagation
            y_pred_prob = model(x)

            # Step 3. Loss Calculation
            loss = F.cross_entropy(y_pred_prob, y, reduction='sum')
            
            # Step 4. Predict label
            y_pred_label = torch.argmax(y_pred_prob, dim=1)
            
            # Step 5. Save real and predicte label
            results_pred.extend(y_pred_label.detach().cpu().numpy())
            results_real.extend(y.detach().cpu().numpy())
            
            # Step 6. eval_loss변수에 mini-batch loss를 누적해서 합산
            eval_loss += loss.item()

    # Step 7. 데이터 한 개당 평균 eval_loss와 accuracy구하기
    avg_eval_loss = eval_loss / len(data_loader.dataset)
    results_pred = np.array(results_pred)
    results_real = np.array(results_real)
    accuracy = np.sum(results_pred == results_real) / len(results_real)
    
    return avg_eval_loss, accuracy

## 4. 매 Epoch에 드는 시간 측정

In [38]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## 5. 학습하기
- <span style = 'font-size:1.2em;line-height:1.5em'>Dataset과 Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [39]:
# torchvision에서도 MNIST데이터를 제공합니다. 
# 이 데이터를 다운 받을 디렉토리(data_path) 존재 여부를 확인하고 존재하지 않으면 생성 
data_path = 'data'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    
# data 변환 방법 선언 (data transform method)
# 아래 예시: numpy형태의 데이터를 받으면 걔를 tensor로 변환해줘!
transform = transforms.Compose([transforms.ToTensor()])

# dataset을 생성 (torchvision에서 제공하는 데이터를 다운 받고, 위의 방법대로 변환)
trn_dset = datasets.MNIST(root=data_path, 
                          train=True, 
                          transform=transform, 
                          download=True)

tst_dset = datasets.MNIST(root=data_path, 
                          train=False, 
                          transform=transform, 
                          download=True)

- <span style = 'font-size:1.2em;line-height:1.5em'>연산을 수행할 device를 설정하기</span>

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- <span style = 'font-size:1.2em;line-height:1.5em'>모델에 대한 객체 생성하기</span>

In [41]:
model = MyNet(dim_in=784, dim_h1=50, dim_h2=100, dim_out=10)
model = model.to(device)

- <span style = 'font-size:1.2em;line-height:1.5em'>학습한 모델을 저장할 directory 생성하기</span>

In [42]:
save_dir = 'models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

- <span style = 'font-size:1.2em;line-height:1.5em'>필요한 hyperparameter값 설정하기</span>

In [43]:
N_EPOCHS = 10
LR = 2e-4
BATCH_SIZE = 2**9

- <span style = 'font-size:1.2em;line-height:1.5em'>Mini-batch를 자동으로 생성할 DataLoader준비하기</span>

In [44]:
trn_loader = DataLoader(trn_dset, batch_size = BATCH_SIZE, shuffle=True, drop_last=False)
tst_loader = DataLoader(tst_dset, batch_size = BATCH_SIZE, shuffle=False, drop_last=False)

- <span style = 'font-size:1.2em;line-height:1.5em'>optimizer 생성하기</span>

In [45]:
my_opt = optim.Adam(model.parameters(), lr = LR)

- <span style = 'font-size:1.2em;line-height:1.5em'>trn_data에 대해서 train()함수를, tst_data에 대해서 evaluate()함수를 반복적으로 호출하면서 모델을 학습</span>
    - <span style = 'font-size:1.2em;line-height:1.5em'>매 epoch마다 학습이 마무리되면, 모델 평가를 진행한다</span>

In [46]:
best_val_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    trn_loss = train(model=model, 
                     data_loader=trn_loader, 
                     optimizer=my_opt, 
                     device=device)
    val_loss, accuracy = evaluate(model=model, 
                                  data_loader=tst_loader, 
                                  optimizer=my_opt, 
                                  device=device)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f'{save_dir}/my_model4.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {trn_loss:.3f} | Test Loss: {val_loss:.3f} | Test Acc: {100*accuracy:.3f}% ')

Epoch: 01 | Time: 0m 10s
	Train Loss: 2.275 | Test Loss: 2.224 | Test Acc: 34.550% 
Epoch: 02 | Time: 0m 11s
	Train Loss: 2.129 | Test Loss: 1.991 | Test Acc: 48.980% 
Epoch: 03 | Time: 0m 10s
	Train Loss: 1.824 | Test Loss: 1.638 | Test Acc: 60.630% 
Epoch: 04 | Time: 0m 11s
	Train Loss: 1.487 | Test Loss: 1.335 | Test Acc: 67.650% 
Epoch: 05 | Time: 0m 10s
	Train Loss: 1.221 | Test Loss: 1.106 | Test Acc: 73.720% 
Epoch: 06 | Time: 0m 10s
	Train Loss: 1.022 | Test Loss: 0.935 | Test Acc: 78.170% 
Epoch: 07 | Time: 0m 11s
	Train Loss: 0.875 | Test Loss: 0.806 | Test Acc: 81.650% 
Epoch: 08 | Time: 0m 10s
	Train Loss: 0.761 | Test Loss: 0.705 | Test Acc: 84.330% 
Epoch: 09 | Time: 0m 10s
	Train Loss: 0.670 | Test Loss: 0.623 | Test Acc: 85.820% 
Epoch: 10 | Time: 0m 10s
	Train Loss: 0.597 | Test Loss: 0.559 | Test Acc: 87.200% 


- <span style = 'font-size:1.2em;line-height:1.5em'><b>Appendix</b> `F.cross_entropy(reduction='sum')`을 하는 이유</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>`train()`, `eval()`함수의 `avg_trn_loss`, `avg_eval_loss` 변수는 각각 데이터 한 개당 평균 loss를 나타내려고 하는 것이다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>이를 구하려면 단일 mini-batch내 각 데이터의 loss를 합산하고, 이를 전체 mini-batch에 대해 전부 계산한뒤, 누적합을 구한다. 그리고 이를 데이터의 개수로 나눠주면, 평균 loss가 나온다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>즉, mini-batch내의 각 데이터의 loss를 합산하여 계산하기 위해 `reduction='sum'`을 사용한 것이다.</span>